In [1]:
import pandas as pd
import numpy as np
import sys
import matplotlib.pyplot as plt
import tensorflow as tf
import os
from glob import glob
sys.path.append('../../src')
sys.path.append('../../')
import classifier.Multiband as multiband


the filtering of the times must be done in the spine, not in each band. Spit all the times then slect the best one. 

In [2]:
# To see if the system recognises the GPU
device = 0
devices = tf.config.experimental.list_physical_devices('GPU')
tf.config.set_visible_devices(devices[device], 'GPU')
tf.config.experimental.set_memory_growth(device=devices[device], enable=True)


2023-07-07 21:32:34.445620: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-07-07 21:32:34.446354: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-07-07 21:32:34.462021: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-07-07 21:32:34.462504: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-07-07 21:32:34.462919: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from S

Find the different folds and train a model using the stored data.

In [3]:
survey = 'Gaia'
path = os.path.join('../02_CreateRecords/Folds/Fold_*/',)
folds = glob(path)
folds.sort()
folds

['../02_CreateRecords/Folds/Fold_1/',
 '../02_CreateRecords/Folds/Fold_2/',
 '../02_CreateRecords/Folds/Fold_3/']

Create folder results

In [4]:
if not os.path.exists('./Results'):
    os.mkdir('./Results')


Define the arguments for all the models.

In [6]:
train_args = {
            'hidden_size_bands':[32, 32, 32],
            'hidden_size_central':[32, 32],
            'fc_layers_bands':[32,32,32],
            'fc_layers_central':[32,32,32], # Neurons of each layer
            'regression_size':[32, 32],#each element is a layer with that size.
            'buffer_size':10000,
            'epochs':5,
            'num_threads':7,
            'batch_size':1024,
            'dropout':0.40,
            'lr':[[1e-3]*2, 1e-3], # [[band1, band2], central]
            'val_steps':50,
            'max_to_keep':0, # Not Used 
            'steps_wait':0, 
            'use_class_weights':False,# Not Used
            'mode' : 'classifier+regression'
            }
loss_weights = {'Class':300.0, 'T_eff':20.0,'Radius':1e0}

callbacks_args = {'patience': 20,
                  'mode':'max',
                  'restore_best_weights':True,
                  'min_delta': 0.001
                 }
train_args_specific={
                    'phys_params': ['T_eff', 'Radius'],
                    'use_output_bands' : True,  # Working
                    'use_output_central' : False, # Not used
                    'use_common_layers' : False, # NOT Working
                    'bidirectional_central' : False,# Working
                    'bidirectional_band' : False,# Not Working
                    'layer_norm_params' : None, # Used to normalyze common layers
                    'use_gated_common' : False, # Working
                    'l1':0.0,
                    'l2':0.0,   
                    'N_skip' : 8, # Cannot be greater than the number of timesteps
                    'use_raw_input_central': True,
                    'train_steps_central' : 2,
                    'print_report' : True,
                    'loss_weights_central' : loss_weights,
                    'callbacks_args':callbacks_args
                    }



In [10]:
base_dir

'../02_CreateRecords/Folds/Fold_1//'

In [ ]:
'../02_CreateRecords/Folds/Fold_1//scalers.pkl'

In [11]:
for fold in folds:
    tf.keras.backend.clear_session()
    # Set the fold path
    base_dir = fold+'/'
    
    # Set the save path for this fold. Create folder if needed
    path_results_fold = fold.replace('../02_CreateRecords', '.').replace('/Folds/', '/Results/')

    if not os.path.exists(path_results_fold):
        os.mkdir(path_results_fold)    

    train_args_specific['save_dir'] = path_results_fold
    train_args_specific['metadata_pre_path'] = os.path.join(fold, 'metadata_preprocess.json')
    train_args_specific['path_scalers'] =  os.path.join(fold,'scalers.pkl')
    # Define the train args
    train_args = {**train_args, **train_args_specific}

    train_files = os.path.join(fold, 'train/*.tfrecord')
    train_files = os.path.join(fold, 'val/*.tfrecord')
    train_files = os.path.join(fold, 'test/*.tfrecord')
    
    new = multiband.Network()    
    new.train(train_args, train_files, val_files, test_files)
    new.train_loop()
    break

./Results/Fold_1/Models/20230707-2140
Start training
13/13 [==============================] - 1s 104ms/step - loss: 7990.5522 - Class_loss: 26.6253 - Pred_Radius_loss: 1.5180 - Pred_T_eff_loss: 0.0728 - Class_CentralAcc: 0.5265 - Class_CentralTop2: 0.7613 - Class_FinalAcc: 0.5336 - Class_FinalTop2: 0.7688 - Class_Final_FScore: 0.3852 - Pred_Radius_Masked_RMSE: 0.9206 - Pred_Radius_Masked_R2: 0.4779 - Pred_T_eff_Masked_RMSE: 0.2174 - Pred_T_eff_Masked_R2: 0.7053
              precision    recall  f1-score   support

         CEP       0.59      0.41      0.49      1367
  DSCT_SXPHE       0.52      0.02      0.04       880
     MIRA_SR       0.88      0.90      0.89      2200
        RRAB       0.44      0.34      0.38      2200
         RRC       0.37      0.74      0.50      2200
       T2CEP       0.00      0.00      0.00       436

    accuracy                           0.53      9283
   macro avg       0.47      0.40      0.38      9283
weighted avg       0.54      0.53      0.49   

/home/ibecker/.cache/pypoetry/virtualenvs/multibandscalable-rnn-2dkdk7iS-py3.8/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/ibecker/.cache/pypoetry/virtualenvs/multibandscalable-rnn-2dkdk7iS-py3.8/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/ibecker/.cache/pypoetry/virtualenvs/multibandscalable-rnn-2dkdk7iS-py3.8/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels w

In [9]:
new.path_scalers

'../02_CreateRecords/Folds/Fold_1//scalers.pkl'